## Deployment Prediksi Kadar NO2 di Banyuwangi 

In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 53.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [streamlit]/9 [streamlit]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
import streamlit as st
import pandas as pd
import joblib
from PIL import Image
import os

# Fungsi untuk memuat model dan scaler dengan caching
@st.cache_resource
def load_model(model_type):
    if model_type == "1_hari":
        model_path = '/content/knn_day2_model.pkl' # Sesuaikan path jika perlu
        scaler_path = '/content/scaler_day2.pkl' # Sesuaikan path jika perlu
    else:  # 3_hari
        model_path = '/content/knn_day5_model.pkl' # Sesuaikan path jika perlu
        scaler_path = '/content/scaler_day5.pkl' # Sesuaikan path jika perlu

    if not os.path.exists(model_path):
        st.error(f"Model file not found at: {model_path}")
        return None, None
    if not os.path.exists(scaler_path):
        st.error(f"Scaler file not found at: {scaler_path}")
        return None, None

    try:
        model = joblib.load(model_path)
        scaler = joblib.load(scaler_path)
        return model, scaler
    except Exception as e:
        st.error(f"Error loading model or scaler: {e}")
        return None, None


st.set_page_config(page_title="Prediksi NO₂ Bangkalan", page_icon="🌫️")

# Menambahkan gambar/logo di bagian atas (opsional)
# Pastikan path gambar benar atau hapus baris ini jika tidak menggunakan gambar
# try:
#     image = Image.open('/content/path_to_your_logo.png') # Ganti dengan path logo Anda
#     st.image(image, use_column_width=True)
# except FileNotFoundError:
#     st.warning("Logo file not found. Please update the image path.")


st.title("🌫️ **Prediksi Kadar NO₂ - Bangkalan**")
st.caption("**Prediksi kadar NO₂ troposfer (mol/m²) menggunakan model KNN berdasarkan data Sentinel-5P.**")

# Pilihan model prediksi dengan gaya berbeda
prediksi_type = st.selectbox(
    "Pilih Jenis Prediksi:",
    ("1_hari", "3_hari"),
    format_func=lambda x: "Prediksi 1 Hari Ke Depan" if x == "1_hari" else "Prediksi 3 Hari Ke Depan",
    help="Pilih model yang digunakan untuk memprediksi kadar NO₂"
)

# Memuat model dan scaler
model, scaler = load_model(prediksi_type)

if model is not None and scaler is not None:
    # Input berdasarkan jenis prediksi
    if prediksi_type == "1_hari":
        col1, col2 = st.columns(2)
        t2 = col1.number_input("NO₂ (t-2)", value=0.000028, format="%.6f", step=0.000001)
        t1 = col2.number_input("NO₂ (t-1)", value=0.000030, format="%.6f", step=0.000001)
    else: # 3_hari
        col1, col2, col3 = st.columns(3)
        t3 = col1.number_input("NO₂ (t-3)", value=0.000025, format="%.6f", step=0.000001)
        t2 = col2.number_input("NO₂ (t-2)", value=0.000028, format="%.6f", step=0.000001)
        t1 = col3.number_input("NO₂ (t-1)", value=0.000030, format="%.6f", step=0.000001)

    # Tombol prediksi
    if st.button("🔮 **Prediksi**", type="primary"):
        if prediksi_type == "1_hari":
            X = pd.DataFrame({'t-1': [t1], 't-2': [t2]})
        else:
            X = pd.DataFrame({'t-3': [t3], 't-2': [t2], 't-1': [t1]})

        try:
            X_scaled = scaler.transform(X)
            y_pred = model.predict(X_scaled)[0]

            st.success("### Hasil Prediksi")

            # Menampilkan hasil prediksi dengan tampilan yang lebih menarik
            # Karena output prediksi 3 hari adalah array, kita perlu menanganinya
            if prediksi_type == "1_hari":
                 st.metric("NO₂ (t+1)", f"{y_pred:.6f} mol/m²")
            else: # 3_hari
                cols = st.columns(3)
                for i in range(len(y_pred)):
                    with cols[i]:
                         st.metric(f"NO₂ (t+{i+1})", f"{y_pred[i]:.6f} mol/m²")

            st.caption("**Prediksi kadar NO₂ yang dihitung menggunakan model KNN.**")

        except Exception as e:
            st.error(f"Error during prediction: {e}")

else:
    st.warning("Model or scaler could not be loaded. Please check the file paths and ensure the files are present.")


# Menambahkan caption untuk menyelesaikan aplikasi
st.divider()
st.caption("Model: KNN Regression | Data: Berdasarkan data Sentinel-5P | Metode Prediksi: KNN")

2025-11-02 17:23:43.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.967 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-02 17:23:43.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()